In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import load_iris, load_wine, load_breast_cancer
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, LeaveOneOut
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 1. Implementacion del Naive Bayes

In [2]:
class NaiveBayes:
    def fit(self, X, y):
        # Almacenar las clases únicas presentes en y
        self.classes = np.unique(y)
        # Inicializar diccionarios para medias, varianzas y probabilidades a priori
        self.mean = {}
        self.var = {}
        self.priors = {}
        
        # Calcular estadísticas para cada clase
        for c in self.classes:
            # Filtrar las muestras que pertenecen a la clase c
            X_c = X[y == c]
            # Calcular la media de cada característica para la clase c
            self.mean[c] = X_c.mean(axis=0)
            # Calcular la varianza de cada característica para la clase c
            # Se añade una pequeña constante para evitar división por cero
            self.var[c] = X_c.var(axis=0) + 1e-9
            # Calcular la probabilidad a priori de la clase c
            self.priors[c] = X_c.shape[0] / X.shape[0]

    def predict(self, X):
        # Predecir la clase para cada muestra en X
        y_pred = [self._predict_instance(x) for x in X]
        return np.array(y_pred)

    def _predict_instance(self, x):
        # Calcular la probabilidad posterior para cada clase
        posteriors = []
        
        for c in self.classes:
            # Obtener el logaritmo del prior de la clase
            prior = np.log(self.priors[c])
            # Calcular la suma de los logaritmos de las probabilidades de las características dado la clase
            class_conditional = np.sum(np.log(self._pdf(c, x)))
            # Calcular el logaritmo de la probabilidad posterior
            posterior = prior + class_conditional
            # Almacenar la probabilidad posterior
            posteriors.append(posterior)
        
        # Retornar la clase con la mayor probabilidad posterior
        return self.classes[np.argmax(posteriors)]
    
    def _pdf(self, class_idx, x):
        # Calcular la función de densidad de probabilidad Gaussiana
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        # Cálculo del numerador de la función de densidad Gaussiana
        numerator = np.exp(- (x - mean) ** 2 / (2 * var))
        # Cálculo del denominador de la función de densidad Gaussiana
        denominator = np.sqrt(2 * np.pi * var)
        # Retornar la probabilidad de x dado la clase
        return numerator / denominator


# 2. Datasets

## 2.1 Wine Dataset

In [3]:
data = load_wine()
X = data.data
y = data.target
target_names = data.target_names

### Hold-out

In [4]:
# Hold-out 70/30 estratificado
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in sss.split(X, y):
    # Dividir los datos en entrenamiento y prueba manteniendo la proporción de clases
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Crear y entrenar el modelo
model = NaiveBayes()
model.fit(X_train, y_train)

# Predecir las etiquetas del conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy Hold-out 70/30: {accuracy:.4f}")

# Generar la matriz de confusión
conf_mat = confusion_matrix(y_test, y_pred)

# Graficar y guardar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Hold-out 70/30')
plt.savefig('wine-holdout.png')
plt.close()


Accuracy Hold-out 70/30: 1.0000


### Validacion cruzada

In [5]:
# Validación cruzada estratificada de 10 particiones
k = 10  # Valor de k elegido anteriormente
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
accuracies = []
conf_mat_total = np.zeros((len(np.unique(y)), len(np.unique(y))), dtype=int)

# Iterar sobre cada fold de la validación cruzada
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    # Dividir los datos en entrenamiento y prueba para el fold actual
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Crear y entrenar el modelo
    model = NaiveBayes()
    model.fit(X_train, y_train)

    # Predecir las etiquetas del conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión y acumularla
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Generar la matriz de confusión y acumularla
    conf_mat = confusion_matrix(y_test, y_pred)
    conf_mat_total += conf_mat

# Calcular la precisión promedio
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy 10-Fold CV: {mean_accuracy:.4f}")

# Graficar y guardar la matriz de confusión acumulada
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat_total, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - 10-Fold CV')
plt.savefig('kfold_wine.png')
plt.close()


Mean Accuracy 10-Fold CV: 0.9778


### Leave One Out

In [6]:
# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
accuracies = []
y_tests = []
y_preds = []

# Iterar sobre cada muestra dejándola como conjunto de prueba
for train_index, test_index in loo.split(X):
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Crear y entrenar el modelo
    model = NaiveBayes()
    model.fit(X_train, y_train)

    # Predecir la etiqueta de la muestra de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión y acumularla
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Almacenar las etiquetas reales y predichas
    y_tests.append(y_test[0])
    y_preds.append(y_pred[0])

# Calcular la precisión promedio
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy Leave-One-Out: {mean_accuracy:.4f}")

# Generar la matriz de confusión
conf_mat = confusion_matrix(y_tests, y_preds)

# Graficar y guardar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Leave-One-Out CV')
plt.savefig('loo_wine.png')
plt.close()


Mean Accuracy Leave-One-Out: 0.9775


### 2.2 Iris Dataset

In [7]:
data = load_iris()
X = data.data
y = data.target
target_names = data.target_names

### Holdout

In [8]:
# Hold-out 70/30 estratificado
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in sss.split(X, y):
    # Dividir los datos en entrenamiento y prueba manteniendo la proporción de clases
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Crear y entrenar el modelo
model = NaiveBayes()
model.fit(X_train, y_train)

# Predecir las etiquetas del conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy Hold-out 70/30: {accuracy:.4f}")

# Generar la matriz de confusión
conf_mat = confusion_matrix(y_test, y_pred)

# Graficar y guardar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Hold-out 70/30')
plt.savefig('iris_holdout.png')
plt.close()


Accuracy Hold-out 70/30: 0.9111


### Validacion Cruzada

In [9]:
# Validación cruzada estratificada de 10 particiones
k = 5  # Valor de k elegido anteriormente
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
accuracies = []
conf_mat_total = np.zeros((len(np.unique(y)), len(np.unique(y))), dtype=int)

# Iterar sobre cada fold de la validación cruzada
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    # Dividir los datos en entrenamiento y prueba para el fold actual
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Crear y entrenar el modelo
    model = NaiveBayes()
    model.fit(X_train, y_train)

    # Predecir las etiquetas del conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión y acumularla
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Generar la matriz de confusión y acumularla
    conf_mat = confusion_matrix(y_test, y_pred)
    conf_mat_total += conf_mat

# Calcular la precisión promedio
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy 10-Fold CV: {mean_accuracy:.4f}")

# Graficar y guardar la matriz de confusión acumulada
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat_total, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - 10-Fold CV')
plt.savefig('kfold_iris.png')
plt.close()


Mean Accuracy 10-Fold CV: 0.9467


### Leave One Out

In [10]:
# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
accuracies = []
y_tests = []
y_preds = []

# Iterar sobre cada muestra dejándola como conjunto de prueba
for train_index, test_index in loo.split(X):
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Crear y entrenar el modelo
    model = NaiveBayes()
    model.fit(X_train, y_train)

    # Predecir la etiqueta de la muestra de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión y acumularla
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Almacenar las etiquetas reales y predichas
    y_tests.append(y_test[0])
    y_preds.append(y_pred[0])

# Calcular la precisión promedio
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy Leave-One-Out: {mean_accuracy:.4f}")

# Generar la matriz de confusión
conf_mat = confusion_matrix(y_tests, y_preds)

# Graficar y guardar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Leave-One-Out CV')
plt.savefig('loo_iris.png')
plt.close()


Mean Accuracy Leave-One-Out: 0.9533


## 2.3 Breast Cancer Dataset

In [11]:
data = load_breast_cancer()
X = data.data
y = data.target
target_names = data.target_names

### Holdout

In [12]:
# Hold-out 70/30 estratificado
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)

for train_index, test_index in sss.split(X, y):
    # Dividir los datos en entrenamiento y prueba manteniendo la proporción de clases
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

# Crear y entrenar el modelo
model = NaiveBayes()
model.fit(X_train, y_train)

# Predecir las etiquetas del conjunto de prueba
y_pred = model.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy Hold-out 70/30: {accuracy:.4f}")

# Generar la matriz de confusión
conf_mat = confusion_matrix(y_test, y_pred)

# Graficar y guardar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Hold-out 70/30')
plt.savefig('breast_holdout.png')
plt.close()


/tmp/ipykernel_128360/2845666160.py:35: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self._pdf(c, x)))


Accuracy Hold-out 70/30: 0.9357


### Validacion cruzada

In [13]:
# Validación cruzada estratificada de 10 particiones
k = 5  # Valor de k elegido anteriormente
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
accuracies = []
conf_mat_total = np.zeros((len(np.unique(y)), len(np.unique(y))), dtype=int)

# Iterar sobre cada fold de la validación cruzada
for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    # Dividir los datos en entrenamiento y prueba para el fold actual
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Crear y entrenar el modelo
    model = NaiveBayes()
    model.fit(X_train, y_train)

    # Predecir las etiquetas del conjunto de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión y acumularla
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Generar la matriz de confusión y acumularla
    conf_mat = confusion_matrix(y_test, y_pred)
    conf_mat_total += conf_mat

# Calcular la precisión promedio
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy 10-Fold CV: {mean_accuracy:.4f}")

# Graficar y guardar la matriz de confusión acumulada
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat_total, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - 10-Fold CV')
plt.savefig('kfold_breast.png')
plt.close()


/tmp/ipykernel_128360/2845666160.py:35: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self._pdf(c, x)))
/tmp/ipykernel_128360/2845666160.py:35: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self._pdf(c, x)))


Mean Accuracy 10-Fold CV: 0.9297


### Leave One Out

In [14]:
# Leave-One-Out Cross-Validation
loo = LeaveOneOut()
accuracies = []
y_tests = []
y_preds = []

# Iterar sobre cada muestra dejándola como conjunto de prueba
for train_index, test_index in loo.split(X):
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Crear y entrenar el modelo
    model = NaiveBayes()
    model.fit(X_train, y_train)

    # Predecir la etiqueta de la muestra de prueba
    y_pred = model.predict(X_test)

    # Calcular la precisión y acumularla
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

    # Almacenar las etiquetas reales y predichas
    y_tests.append(y_test[0])
    y_preds.append(y_pred[0])

# Calcular la precisión promedio
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy Leave-One-Out: {mean_accuracy:.4f}")

# Generar la matriz de confusión
conf_mat = confusion_matrix(y_tests, y_preds)

# Graficar y guardar la matriz de confusión
plt.figure(figsize=(8,6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names, yticklabels=target_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Confusion Matrix - Leave-One-Out CV')
plt.savefig('loo_breast.png')
plt.close()


/tmp/ipykernel_128360/2845666160.py:35: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self._pdf(c, x)))
/tmp/ipykernel_128360/2845666160.py:35: RuntimeWarning: divide by zero encountered in log
  class_conditional = np.sum(np.log(self._pdf(c, x)))


Mean Accuracy Leave-One-Out: 0.9332
